# 1.0 Import

In [12]:
#importando as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

## Load DataSet

In [14]:
df = pd.read_csv('../data/b3_bovespa.csv')

# Select only one share to predict

In [15]:
# Filter share_data
df_itau = df[df['sigla_acao']=='ITUB4']

In [16]:
df_itau.columns

Index(['data_pregao', 'sigla_acao', 'nome_acao', 'preco_abertura',
       'preco_maximo', 'preco_minimo', 'preco_fechamento', 'qtd_negocios',
       'volume_negocios'],
      dtype='object')

In [17]:
df_itau.head()

,data_pregao,sigla_acao,nome_acao,preco_abertura,preco_maximo,preco_minimo,preco_fechamento,qtd_negocios,volume_negocios
197,2019-01-02,ITUB4,ITAUUNIBANCO,35.44,37.14,35.35,37.00,25581700,9
553,2019-01-03,ITUB4,ITAUUNIBANCO,36.75,37.61,36.45,37.61,21938600,8
901,2019-01-04,ITUB4,ITAUUNIBANCO,37.22,37.71,36.75,36.98,24873500,9
1250,2019-01-07,ITUB4,ITAUUNIBANCO,36.80,37.50,36.75,37.05,16901100,6
1617,2019-01-08,ITUB4,ITAUUNIBANCO,37.18,37.57,36.56,37.52,19556300,7
